### Heroes Of Pymoli Data Analysis
* Of the 1163 active players, the vast majority are male (84%). There also exists, a smaller, but notable proportion of female players (14%).

* Our peak age demographic falls between 20-24 (44.8%) with secondary groups falling between 15-19 (18.60%) and 25-29 (13.4%).  
-----

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

## Player Count

In [2]:
#Count the number of players using the length of and value count of SN column
Total_Players = len(purchase_data["SN"].value_counts())
Total_Players

576

In [3]:
#Create a dataframe with the number of players 
summary_table = pd.DataFrame({"Total Players": [Total_Players]                          
                              })
summary_table

,Total Players
0,576


## Purchasing Analysis (Total)

In [4]:
#Calculate the average price with the .mean() function
Average_Price = purchase_data["Price"].mean()
Average_Price 


3.050987179487176

In [5]:
#Calculate the number of purchase with the len() function
Number_of_Purchases = len(purchase_data)

Number_of_Purchases

780

In [6]:
#Calculate the total revenue with the .sum() on the "Price" column
Total_Revenue = purchase_data["Price"].sum()
Total_Revenue

2379.77

In [7]:
# get the number of unique items from "item ID" and passing the len() function
Number_of_unique_items = len(purchase_data["Item ID"].unique())
Number_of_unique_items

183

In [8]:
#Create a dataframe with the results from the calculations above
summary_table = pd.DataFrame({"Number of Unique Items": [Number_of_unique_items],
                              "Average Price": [Average_Price],
                              "Number of Purchases": [Number_of_Purchases],
                              "Total Revenue": [Total_Revenue]})
summary_table

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,3.050987,780,2379.77


## Gender Demographics

In [9]:
#drop the duplicates from the purchase_data dataframe 
drop_gender_df = purchase_data.drop_duplicates(subset='SN',keep='first')

#reading the head of the new cleaned dataframe
drop_gender_df.head()

# get the number of values per gender using value_counts() function
Total_count = drop_gender_df['Gender'].value_counts()
Total_count




Male                     484
Female                    81
Other / Non-Disclosed     11
Name: Gender, dtype: int64

In [10]:
# Calcuate the percentage for each gender 
Percentage_Count = (Total_count/Total_Players)*100
Percentage_Count

Male                     84.027778
Female                   14.062500
Other / Non-Disclosed     1.909722
Name: Gender, dtype: float64

In [11]:
# Creating a new DataFrame using the count and percentage for each gender
summary_table = pd.DataFrame({"Total Count": Total_count,
                                    "Percentage Count": Percentage_Count})
summary_table.head()

,Total Count,Percentage Count
Male,484,84.027778
Female,81,14.062500
Other / Non-Disclosed,11,1.909722



## Purchasing Analysis (Gender)

In [60]:
#calculating number of players per gender and sum of price 
Gender_Count_Price_Sum = purchase_data.groupby('Gender').agg({"Gender" : "count",
                                                        "Price" : "sum"})


In [61]:
#Calculating average purchase price per gender

Gender_Count_Price_Sum ["Average Purchase Price"] = (Gender_Count_Price_Sum ["Price"]/Gender_Count_Price_Sum["Gender"]).map("${:.2f}".format)


In [62]:
#Formatting Gender_Count_Price_Sum 
Gender_Count_Price_Sum ["Price"] = Gender_Count_Price_Sum ["Price"].map("${:.2f}".format)

In [63]:
#Renaming Gender_Count_Price_sum
Gender_Count_Price_Sum  = Gender_Count_Price_Sum.rename(columns = {"Gender" : "Purchase Count","Price": "Total Purchase Value"})
Gender_Count_Price_Sum  = Gender_Count_Price_Sum [["Purchase Count","Average Purchase Price","Total Purchase Value"]]


In [64]:
#Calculating sum_of_Price and formating 
Sum_of_Price = purchase_data.groupby('SN').agg({"Price" : "sum"})

#Merging Sum_of_Price with pruchase_data
Sum_of_Price = pd.merge(Sum_of_Price,purchase_data[['SN','Gender']],how='left',left_on=['SN'],right_on=['SN'])
#Droping the dyplicates from the new dataframe in the 'SN' column
Sum_of_Price = Sum_of_Price.drop_duplicates('SN') 


In [65]:
#Calculating average price person and formating 
Average_Price_per_Person = Sum_of_Price.groupby('Gender').agg({"Price" : "mean"})

#Formatting the "Price" colum
Average_Price_per_Person["Price"] = Average_Price_per_Person["Price"].map("${:.2f}".format)

#Renaming the price column
Average_Price_per_Person = Average_Price_per_Person.rename(
   columns = {"Price" : "Average Total Purchase per Person"})


In [66]:
#merging Gender_Count_Price_Sum and Average_Price_per_Person
Gender_Count_Price_Sum = pd.merge(Gender_Count_Price_Sum, Average_Price_per_Person,
                             how = 'left', left_on = ['Gender'], right_on = ['Gender'])

In [67]:
#print result
Gender_Count_Price_Sum

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

In [83]:
#Creating bins for age
bins = [0, 9, 14, 19, 24, 29, 34, 39, purchase_data["Age"].max()]
#creating labels for the bins
group_labels =["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

In [84]:
#Droping duplicates in Age, SN, Gender
uniqueplayer = purchase_data[["Age","SN","Gender"]].drop_duplicates('SN') 




In [85]:
#Player count by SN
player_count =uniqueplayer['SN'].count()


In [86]:
#Slicing the data and putting it into bins 
uniqueplayer["Age Group"] = pd.cut(uniqueplayer["Age"], bins, labels=group_labels)

#Setting a Age Group grouby and calculating count of gender

age_player = uniqueplayer.groupby("Age Group").agg({"Gender" : "count"})

#Renaming the count of gender

age_player = age_player.rename(columns = {"Gender" : "Total Count"})

#Calculating the percentage of player
age_player["Percentage of Players"] = (age_player["Total Count"]/player_count*100).map("{:.2f}".format)

In [87]:
#Deleting the index name
del age_player.index.name

#Printing age_player
age_player


,Total Count,Percentage of Players
<10,17,2.95
10-14,22,3.82
15-19,107,18.58
20-24,258,44.79
25-29,77,13.37
30-34,52,9.03
35-39,31,5.38
40+,12,2.08


## Purchasing Analysis (Age)

In [42]:
#Creating age bins 
bins = [0, 10, 15, 20,
         25, 30, 35, 40, 50]

#Setting labels for the age bins 
group_labels = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34",
                "35-39", "40+"]




In [43]:
#Creating a copy of purchase data 
purchase_data_copy= purchase_data.copy()

In [44]:
#Slicing the data and putting it into bins by Age Group
purchase_data_copy["Age Group"] = pd.cut(purchase_data["Age"], bins, labels=group_labels)

In [45]:
#Calculating count and price sum 

age_count_sum = purchase_data_copy.groupby('Age Group').agg({'Item ID' : 'count',
                                                                         'Price' : 'sum'})
                                                                         

In [46]:
#Renaming more columns names 
age_count_sum  = age_count_sum.rename(columns = {"Item ID" : "Purchase Count",
                                                          "Price" : "Total Purchase Value"})

In [47]:
#Calculating average purchase price
age_count_sum ["Average Purchase Price"] = age_count_sum ['Total Purchase Value']/age_count_sum ['Purchase Count']

In [48]:
#Calculating average price per person and renaming column 
average_purchase_per_person = purchase_data_copy.groupby(['Age Group','SN']).agg({'Price' : 'sum'})


average_purchase_per_person = average_purchase_per_person.groupby('Age Group').agg({'Price':'mean'})
#Renaming the price column
average_purchase_per_person =average_purchase_per_person.rename(columns = {"Price" : "Average Total Purchase per Person"})


In [49]:
#merging age_count_sum with average_purchase_per_person 
age_count_sum = pd.merge(age_count_sum , average_purchase_per_person, how='left',
                              left_on=['Age Group'],right_on=['Age Group'])



In [50]:
#Setting the column of dataset 
age_count_sum = age_count_sum[['Purchase Count', 'Average Purchase Price',
                                          'Total Purchase Value','Average Total Purchase per Person']]

In [51]:
#formatting age_count_sum 
age_count_sum['Average Purchase Price'] = age_count_sum['Average Purchase Price'].map("${:.2f}".format)
age_count_sum['Total Purchase Value'] = age_count_sum['Total Purchase Value'].map("${:.2f}".format)
age_count_sum['Average Total Purchase per Person'] = age_count_sum['Average Total Purchase per Person'].map("${:.2f}".format)
age_count_sum = age_count_sum.reindex(["10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+","<10"])
del age_count_sum.index.name


In [52]:
#printing age_count_sum
age_count_sum

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Total Purchase per Person
10-14,54,$2.90,$156.60,$3.82
15-19,200,$3.11,$621.56,$4.14
20-24,325,$3.02,$981.64,$4.23
25-29,77,$2.88,$221.42,$3.75
30-34,52,$2.99,$155.71,$4.21
35-39,33,$3.40,$112.35,$4.32
40+,7,$3.08,$21.53,$3.08
<10,32,$3.40,$108.96,$4.54


## Top Spenders

In [38]:
#setting a groupby and calculating
top_spender = purchase_data.groupby(['SN']).agg({"SN" : "count",  "Price" : ['mean','sum']})

In [39]:
#setting columns, renaming and sorting
top_spender.columns = top_spender.columns.get_level_values(1)
top_spender =top_spender.rename(columns = {"count" : "Purchase Count", 'mean' : 'Average Purchase Price',
                                                          "sum" : "Total Purchase Value"})
top_spender = top_spender.sort_values('Total Purchase Value', ascending = False)

In [40]:
#formatting 
top_spender['Average Purchase Price']=top_spender['Average Purchase Price'].map("${:.2f}".format)

In [41]:
#formating and printing 
top_spender['Total Purchase Value']=top_spender['Total Purchase Value'].map("${:.2f}".format)
top_spender.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

In [19]:
#setting groupby by Item Id and Item name and calculating
popular_items = purchase_data.groupby(['Item ID','Item Name']).agg({"Item ID" : "count",
                                                                    "Price" : "mean"})


In [20]:
#renaming columns
popular_items = popular_items.rename(columns = {"Price" : "Item Price", "Item ID" : "Purchase Count"})

In [21]:
#formating
popular_items["Total Purchase Value"] = (popular_items["Item Price"]*popular_items["Purchase Count"]).map("${:.2f}".format)

In [22]:
#sort_values of Puchase Count in ascending way
popular_items =popular_items.sort_values('Purchase Count', ascending = False)

In [23]:
#printing
popular_items.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,4.23,$50.76
145,Fiery Glass Crusader,9,4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,3.53,$31.77
82,Nirvana,9,4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,1.02,$8.16


## Most Profitable Items

In [33]:
#replacing 
popular_items['Total Purchase Value'] = popular_items['Total Purchase Value'].str.replace('$', '')


In [34]:
#setting the type of popular_items
popular_items['Total Purchase Value'] = popular_items['Total Purchase Value'].astype(float)

In [35]:
#sorting values in ascending way
popular_items = popular_items.sort_values('Total Purchase Value', ascending = False)

In [36]:
#formating
popular_items['Total Purchase Value'] = popular_items['Total Purchase Value'].map("${:.2f}".format)

In [37]:
#printing popular_items 
popular_items.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,4.23,$50.76
82,Nirvana,9,4.90,$44.10
145,Fiery Glass Crusader,9,4.58,$41.22
92,Final Critic,8,4.88,$39.04
103,Singed Scalpel,8,4.35,$34.80
